In [ ]:
# 변수의 shape, type, value 확인
def p(var,_name="") :
    if _name != "" : print(f'<<{_name}>>')
    if type(var)!=type([]):
        try:
            print(f'Shape:{var.shape}')
        except :
            pass
    print(f'Type: {type(var)}')
    print(f'Values: {var}')

def pst(_x,_name=""):
    print(f'<<{_name}>> Shape{_x.shape}, {type(_x)}')
def ps(_x,_name=""):
    print(f'<<{_name}>> Shape{_x.shape}')

In [ ]:
!pip install -q accelerate
!pip install -q langchain huggingface-hub faiss-cpu pypdf
!pip install -q langchain_community
!pip install -U langchain-huggingface -q
!pip install flash-attn --no-build-isolation -q
!pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import os
from google.colab import userdata

# Colab Secrets에서 토큰 읽어오기
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

## Node.js 및 MCP 서버 설정

In [ ]:
# Remove existing Node.js
!apt-get remove -y nodejs npm
!apt-get autoremove -y

# Install NVM
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.7/install.sh | bash
%env NVM_DIR=/root/.nvm
!source /root/.nvm/nvm.sh && nvm install 20 && nvm use 20

# Verify Node.js and npm versions
!node -v  # Should output ~v20.x.x
!npm -v   # Should output ~10.x.x

# Install MCP server
!npm install -g @modelcontextprotocol/server-filesystem

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'npm' is not installed, so not removed
Package 'nodejs' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16555  100 16555    0     0  53805      0 --:--:-- --:--:-- --:--:-- 53925
=> Downloading nvm from git to '/root/.nvm'
=> Cloning into '/root/.nvm'...
remote: Enumerating objects: 382, done.
remote: Counting objects: 100% (382/382), done.
remote: Compressing objects: 100% (325/325), done.
remote: Total 382 (delta 43), reused 179 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (382/382), 385.06 KiB | 25.67

## MCP 서버 실행

In [ ]:
import subprocess
import time
import os

# Verify Node.js installation (from previous fix)
!node -v  # Should output ~v20.x.x
!npm -v   # Should output ~10.x.x

# Start MCP server
mcp_process = subprocess.Popen(
    ["npx", "@modelcontextprotocol/server-filesystem", "/content"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
print("MCP 서버 실행 중...")
time.sleep(5)  # Wait for server to start

# Check server status
try:
    stdout, stderr = mcp_process.communicate(timeout=5)
    if "Secure MCP Filesystem Server running on stdio" in stderr:
        print("MCP 서버 정상 실행: Secure MCP Filesystem Server running on stdio")
        print(f"허용된 디렉토리: {stderr.split('Allowed directories: ')[-1].strip()}")
    elif stderr:
        print(f"MCP 서버 오류: {stderr}")
    else:
        print(f"MCP 서버 출력: {stdout}")
except subprocess.TimeoutExpired:
    print("MCP 서버가 백그라운드에서 실행 중입니다. 상태 확인을 위해 테스트 명령을 실행합니다.")

    # Test MCP server with a simple command
    test_process = subprocess.Popen(
        ["npx", "@modelcontextprotocol/server-filesystem", "/content"],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    test_stdout, test_stderr = test_process.communicate(input=json.dumps({"command": "list_dir /content"}))
    if "Secure MCP Filesystem Server running on stdio" in test_stderr:
        print("MCP 서버 정상 실행 확인됨")
    elif test_stderr:
        print(f"MCP 서버 테스트 오류: {test_stderr}")
    else:
        print(f"MCP 서버 테스트 출력: {test_stdout}")

v20.19.0
10.8.2
MCP 서버 실행 중...
MCP 서버 정상 실행: Secure MCP Filesystem Server running on stdio
허용된 디렉토리: [ '/content' ]


## LLM, Tool 정의

## Local LLM 설치

In [ ]:
from google.colab import userdata
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face token
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# Model configuration
# meta-llame 처음 사용시 hugging face 에서 사용 승인 필요: 검토에 시간이 필요함
# model_id = "meta-llama/Llama-3.1-8B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "google/gemma-2-9b-it"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HF_TOKEN"])
print("모델 로딩 중...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    token=os.environ["HF_TOKEN"]
)
print("모델 로딩 완료.")
torch.cuda.empty_cache()
model.to("cuda:0")
print(f"Model device: {model.device}")

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

모델 로딩 중...


...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

모델 로딩 완료.
Model device: cuda:0


## gdrive mount

In [ ]:
## colab VM으로 직접 file upload 하기
# from google.colab import files
# uploaded = files.upload()  # Upload manual.txt, report.pdf, data.xlsx

In [ ]:
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


## MCP Interface

In [ ]:
!pip install PyPDF2 -q

In [ ]:
import json
import pandas as pd
import PyPDF2

path = "/content/gdrive/MyDrive/Colab Notebooks/datasets/LLMs/manuals"

## MCP 서버 Interface
def query_mcp_server(command: str) -> str:
    """
    Send a command to the MCP server and return the response.
    """
    process = subprocess.Popen(
        ["npx", "@modelcontextprotocol/server-filesystem", path],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    stdout, stderr = process.communicate(input=json.dumps({"command": command}))
    if stderr and "Secure MCP Filesystem Server running on stdio" not in stderr:
        return f"MCP 서버 오류: {stderr}"
    return stdout.strip() or "파일 내용이 비어 있습니다."

## File read tool
def read_file_fallback(file_path: str) -> str:
    """
    Read text, PDF, or Excel files using Python if MCP fails.
    """
    try:
        if file_path.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        elif file_path.endswith('.pdf'):
            with open(file_path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                return text
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
            return df.to_string()
        else:
            return f"지원되지 않는 파일 형식: {file_path}"
    except Exception as e:
        return f"파일 읽기 오류: {str(e)}"

## File 기반 Query 처리기
def process_file_with_llm(file_path: str, question: str) -> str:
    """
    Read file via MCP (text) or Python fallback (PDF, Excel) and process with LLM.
    """
    # Determine file type
    is_text = file_path.endswith('.txt')

    # Try MCP for text files
    file_content = ""
    if is_text:
        command = f"read_file {file_path}"
        file_content = query_mcp_server(command)
        if "MCP 서버 오류" in file_content or "파일 내용이 비어 있습니다" in file_content:
            print("MCP 서버 실패, Python으로 파일 읽기 시도...")
            file_content = read_file_fallback(file_path)
    else:
        # Use Python for non-text files
        print(f"{file_path}은 텍스트 파일이 아니므로 Python으로 처리...")
        file_content = read_file_fallback(file_path)

    # Create prompt
    prompt = f"다음 파일 내용을 기반으로 질문에 답변하세요:\n\n{file_content}\n\n질문: {question}"

    # Move inputs to GPU
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    print(f"Input devices: {[f'{k}: {v.device}' for k, v in inputs.items()]}")

    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.3
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
    print(">>> 'quality_checker' 노드 및 엣지 추가 완료. <<<\n")

## File 기반 Query test

In [ ]:
# PDF file 기반 Query test
response = process_file_with_llm(
    file_path= path+"/규격서_세포칩용 포토리소장비.pdf",
    question="내용을 요약해줘."
)
print("\nPDF File Response:")
print(response)

/content/gdrive/MyDrive/Colab Notebooks/datasets/LLMs/manuals/규격서_세포칩용 포토리소장비.pdf은 텍스트 파일이 아니므로 Python으로 처리...
Input devices: ['input_ids: cuda:0', 'attention_mask: cuda:0']

PDF File Response:
다음 파일 내용을 기반으로 질문에 답변하세요:

규  격  서
 구매 장비명 : 미세패턴형성장치
 세부품명번호 : 2321119401
 조달정 물품 분류 번호: 23211194
 조당철 물품 식별 번호: 21613717
구 분 상 세 내 역 수 량 비 고
 1. Feature :1) Photo Lithography 공정을 수행하는 시스템으로서 감광막에 자외선
  (UV)을 직접 조사하여 반응 Wafer 및 기타 Substrate 위의 다양한 물
  질에 Pattern 을 형성시키는 것을 목적으로 한다. 
2) 옵션으로 , Nanoimprint kit 를 추가하면 나노 패턴 스탬프로 고해상도 
   패턴 구현이 가능하다 .1
 2. Specification :
 가. Main System
MODEL: MDA-400S
  ◦ 일반사양
   - Dimension: 1110*1000*850
   - weight: 250kg (main)
   - working space size : 2200(W) X 2000(D) X 2300(H)
   - Manual & Pneumatic Control system
   - Sample Size : ∼ 6" wafer        
   - Mask Holder size : multi ∼ 7”x 7”  
   - UV Exposure Light source with Power supply
   - Dual CCD zoom microscope
   - Alignment Tooling module with X,Y,Z and Theta motion 
   - Wedge Error Compensati

In [ ]:
# Text file test
response = process_file_with_llm(
    file_path= path+"manual.txt",
    question="포토리소그래피 관련 내용을 요약해줘."
)
print("\nText File Response:")
print(response)

MCP 서버 실패, Python으로 파일 읽기 시도...

오류 발생: name 'tokenizer' is not defined


In [ ]:
# Excel file test
response = process_file_with_llm(
    file_path= path+"/data.xlsx",
    question="첫 번째 시트의 데이터를 설명해줘."
)
print("\nExcel File Response:")
print(response)

## Folder 접근 및 정리   
### low-level coding으로 접근

In [ ]:
query_mcp_server("list_dir /content")

'파일 내용이 비어 있습니다.'

In [ ]:
import shutil

def organize_folders():
    """
    Organize files in /content into folders by extension using MCP and Python.
    """
    # List files using MCP
    file_list = query_mcp_server("list_dir /content")
    if "MCP 서버 오류" in file_list:
        print("MCP list_dir 실패, Python으로 파일 목록 가져오기...")
        file_list = [f for f in os.listdir("/content") if os.path.isfile(os.path.join("/content", f))]
    else:
        try:
            file_list = json.loads(file_list).get("files", [])
        except json.JSONDecodeError:
            file_list = [f for f in os.listdir("/content") if os.path.isfile(os.path.join("/content", f))]

    # Create folders
    folders = {
        ".txt": "/content/txt_files",
        ".pdf": "/content/pdf_files",
        ".xlsx": "/content/excel_files"
    }
    for folder in folders.values():
        os.makedirs(folder, exist_ok=True)

    # Move files
    for file_name in file_list:
        src_path = os.path.join("/content", file_name)
        for ext, dest_folder in folders.items():
            if file_name.endswith(ext):
                dest_path = os.path.join(dest_folder, file_name)
                try:
                    shutil.move(src_path, dest_path)
                    print(f"이동: {file_name} -> {dest_folder}")
                except Exception as e:
                    print(f"파일 이동 오류: {file_name}, {str(e)}")
                break

In [ ]:
# Organize folders
print("폴더 정리 시작...")
organize_folders()
print("폴더 정리 완료.")

폴더 정리 시작...
폴더 정리 완료.
